In [1]:
import os
from argparse import ArgumentParser
from dataclasses import dataclass
from enum import Enum
from typing import List
from tqdm import tqdm
import numpy as np
import hsfs

from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import IntegerType, StringType, StructField, StructType

Starting Spark application


ID,Application ID,Kind,State,Spark UI,Driver log
45,application_1646303173729_0004,pyspark,idle,Link,Link


SparkSession available as 'spark'.


In [2]:
connection = hsfs.connection()
fs = connection.get_feature_store()

# Storage connector to s3
sc = fs.get_storage_connector("experiment-s3")

Connected. Call `.close()` to terminate connection gracefully.

In [3]:
sc.prepare_spark()

In [4]:
s3_directory = "s3a://" + sc.bucket + "/axel_experiments"

In [6]:
rng = np.random.default_rng(1337)
SECONDS_IN_DAY = 60 * 60 * 24
SECONDS_IN_MONTH = SECONDS_IN_DAY * 30

In [7]:
LEFT_SCHEMA = StructType(
    [
        StructField("id", IntegerType(), False),
        StructField("ts", IntegerType(), False),
        StructField("label", StringType(), False),
    ]
)

RIGHT_SCHEMA = StructType(
    [
        StructField("id", IntegerType(), False),
        StructField("ts", IntegerType(), False),
        StructField("value", StringType(), False),
    ]
)

In [8]:
@dataclass
class DistributionConfiguration:
    mean: float
    sd: float


class TimestampDistribution(Enum):
    Normal = "normal"
    Uniform = "uniform"

    def __str__(self) -> str:
        return self.value


In [9]:
def limit_value(value, min_v, max_v):
    return min(max(min_v, value), max_v)

In [10]:
def generate_left_table(unique_ids: int, max_timestamp: int) -> DataFrame:
    # Create RDD of all initial RDDs
    id_rdd = spark.sparkContext.parallelize([id for id in range(1, unique_ids + 1)])
    left_rdd = id_rdd.map(lambda id : (
        id, 
        rng.integers(low=0, high=max_timestamp).item(),
        "{}".format(id),
    ))
    return left_rdd.toDF(LEFT_SCHEMA)

In [11]:
def generate_right_table(
    unique_ids: int,
    max_timestamp: int,
    events_per_id_confs: List[DistributionConfiguration],
    timestamp_distributions: List[TimestampDistribution],
):
    id_rdd = spark.sparkContext.parallelize([id for id in range(1, unique_ids + 1)])
    def mapping_func(id):
        # Pick a event generation conf at random
        events_per_id_conf_idx = rng.integers(
            low=0, high=len(events_per_id_confs), size=1
        )[0]
        events_per_id_conf = events_per_id_confs[events_per_id_conf_idx]
        events = max(rng.normal(loc=events_per_id_conf.mean, scale=events_per_id_conf.sd, size=1)[0].astype(int), 0)  # type: ignore

        # Pick a timestamp distribution configuration
        timestamp_distribution_idx = rng.integers(
            low=0, high=len(timestamp_distributions), size=1
        )[0]
        timestamp_distribution = timestamp_distributions[timestamp_distribution_idx]

        if timestamp_distribution is TimestampDistribution.Uniform:
            timestamps = rng.integers(low=0, high=max_timestamp, size=events).tolist()
        elif timestamp_distribution is TimestampDistribution.Normal:
            mean = rng.uniform(low=0, high=max_timestamp, size=1)[0]
            sd = rng.uniform(low=SECONDS_IN_DAY * 5, high=SECONDS_IN_DAY * 15)
            timestamps = rng.normal(mean, sd, events).astype(int).tolist()  # type: ignore
        else:
            raise Exception(
                "Invalid timestamp distribution: {}".format(timestamp_distribution)
            )
        return [
            (id, limit_value(timestamp, 0, max_timestamp  - 1), "{}".format(id))
            for timestamp in timestamps
        ]

    right_rdd = id_rdd.flatMap(mapping_func)
    return right_rdd.toDF(RIGHT_SCHEMA)

"""
generate_right_table(1_000_000, SECONDS_IN_MONTH * 12, [
    DistributionConfiguration(20, 2),
    DistributionConfiguration(80, 8),
], [TimestampDistribution.Uniform, TimestmapDistribution.Normal]).count()
"""

'\ngenerate_right_table(1_000_000, SECONDS_IN_MONTH * 12, [\n    DistributionConfiguration(20, 2),\n    DistributionConfiguration(80, 8),\n], [TimestampDistribution.Uniform, TimestmapDistribution.Normal]).count()\n'

In [12]:
def save(ids, max_ts, left: DataFrame, right: DataFrame):
    directory = s3_directory + "/raw/{}-{}".format(ids, max_ts)
    left.write.parquet(directory + "/left.parquet", mode="overwrite")
    # left.to_csv(directory + "/debug_left.csv")
    right.write.parquet(directory + "/right.parquet", mode="overwrite")
    # right.to_csv(directory + "/debug_right.csv")

In [13]:
UNIQUE_IDS = [10_000, 100_000, 1_000_000, 10_000_000]
INTERVALS = [
    (SECONDS_IN_MONTH * 12, "1_year"),
]
FEATURE_UPDATES = [
    DistributionConfiguration(20, 2),
    DistributionConfiguration(80, 8),
]
NO_FEATURES = [1, 2]
DISTRIBUTIONS = [TimestampDistribution.Normal, TimestampDistribution.Uniform]

In [14]:
def generate_all():
    progress = tqdm(total=sum(UNIQUE_IDS))
    for interval in INTERVALS:
        for unique_ids in UNIQUE_IDS:
            left = generate_left_table(unique_ids, interval[0])
            right = generate_right_table(
                unique_ids, interval[0], FEATURE_UPDATES, DISTRIBUTIONS
            )
            save(
                unique_ids,
                interval[1],
                left,
                right,
            )
            progress.update(unique_ids)

In [15]:
generate_all()

100%|##########| 11110000/11110000 [1:53:14<00:00, 451.61it/s]